In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotVisibleException
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import datetime
import time
import os
from requests import get
import pandas as pd
import json

In [ ]:
#음식점 이름 -> 페이지 id 알아내기

location = "회기"
res_info = pd.read_csv("./dongdaemoon.csv", engine = "python", encoding = "CP949")
res_info = res_info.fillna("")

hoegi = res_info[res_info["소재지(지번)"].str.contains(location)]
rest_name = hoegi["업소명"]
rest_name.head(10)

In [ ]:
id_list = list(rest_name)

In [ ]:
names, addresses, menu_lists = [], [], []

for id_ in id_list:
    
    #음식점 메인 페이지 request
    url = "https://m.place.naver.com/restaurant/" + id_ + "/home"
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    main_req = get(url, headers = headers)
    soup = BeautifulSoup(main_req.content.decode('utf-8','replace'))
    print(soup)
    #음식점 이름, 주소 크롤링
    place_name = soup.select_one("._3XamX").get_text()
    address = soup.select_one("._2yqUQ").get_text()

    #메뉴 리스트 페이지 request
    menu_url = url.replace("/home", "") + "/menu/list"
    menu_req = get(menu_url)
    menu_soup = BeautifulSoup(menu_req.content.decode("utf-8", "replace"))
    
    #메뉴 리스트 크롤링
    menu_list = menu_soup.select("._3yfZ1")
    for (idx, menu) in enumerate(menu_list):
        menu_list[idx] = menu.get_text()

    #리스트 저장
    names.append(place_name)
    addresses.append(address)
    menu_lists.append(menu_list)

#데이터 프레임으로 저장
df = pd.DataFrame()
df["식당명"] = names
df["주소"] = addresses
df["메뉴"] = menu_lists

In [ ]:
df